In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#Setting the training and test path for the images
TrainingImagePath='../Face Images/Final Training Images'
TestingImagePath='../Face Images/Final Testing Images'


#adding final manipulations to all data
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)


test_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)


training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(100,100),
        batch_size=32,
        class_mode='categorical')


test_set = test_datagen.flow_from_directory(
        TestingImagePath,
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 1712 images belonging to 43 classes.
Found 697 images belonging to 43 classes.


{'Akhmed_Zakayev': 0,
 'Amelia_Vega': 1,
 'Ana_Palacio': 2,
 'Anders_Fogh_Rasmussen': 3,
 'Andre_Agassi': 4,
 'Angela_Bassett': 5,
 'Clare': 6,
 'Dale_Earnhardt_Jr': 7,
 'Edmund_Stoiber': 8,
 'Gregg_Popovich': 9,
 'Jacques_Rogge': 10,
 'Jimmy_Carter': 11,
 'Juan_Pablo_Montoya': 12,
 'Kim_Clijsters': 13,
 'Lance_Armstrong': 14,
 'Lleyton_Hewitt': 15,
 'Maria_Soledad_Alvear_Valenzuela': 16,
 'Mark_Philippoussis': 17,
 'Megawati_Sukarnoputri': 18,
 'Michael_Bloomberg': 19,
 'Nestor_Kirchner': 20,
 'Paul_Bremer': 21,
 'Pervez_Musharraf': 22,
 'Rick_Perry': 23,
 'Tommy_Thompson': 24,
 'Vaclav_Havel': 25,
 'Vincent_Brooks': 26,
 'face1': 27,
 'face10': 28,
 'face11': 29,
 'face12': 30,
 'face13': 31,
 'face14': 32,
 'face15': 33,
 'face16': 34,
 'face2': 35,
 'face3': 36,
 'face4': 37,
 'face5': 38,
 'face6': 39,
 'face7': 40,
 'face8': 41,
 'face9': 42}

Creating a table of all the faces with ids. This will be used for identifying the correct result.

In [16]:
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag for future reference
PersonMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    PersonMap[faceValue]=faceName

# Saving the face map for future reference
import pickle
with open("facesMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(PersonMap, fileWriteStream)

# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",PersonMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(PersonMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'Akhmed_Zakayev', 1: 'Amelia_Vega', 2: 'Ana_Palacio', 3: 'Anders_Fogh_Rasmussen', 4: 'Andre_Agassi', 5: 'Angela_Bassett', 6: 'Clare', 7: 'Dale_Earnhardt_Jr', 8: 'Edmund_Stoiber', 9: 'Gregg_Popovich', 10: 'Jacques_Rogge', 11: 'Jimmy_Carter', 12: 'Juan_Pablo_Montoya', 13: 'Kim_Clijsters', 14: 'Lance_Armstrong', 15: 'Lleyton_Hewitt', 16: 'Maria_Soledad_Alvear_Valenzuela', 17: 'Mark_Philippoussis', 18: 'Megawati_Sukarnoputri', 19: 'Michael_Bloomberg', 20: 'Nestor_Kirchner', 21: 'Paul_Bremer', 22: 'Pervez_Musharraf', 23: 'Rick_Perry', 24: 'Tommy_Thompson', 25: 'Vaclav_Havel', 26: 'Vincent_Brooks', 27: 'face1', 28: 'face10', 29: 'face11', 30: 'face12', 31: 'face13', 32: 'face14', 33: 'face15', 34: 'face16', 35: 'face2', 36: 'face3', 37: 'face4', 38: 'face5', 39: 'face6', 40: 'face7', 41: 'face8', 42: 'face9'}

 The Number of output neurons:  43


In [5]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

epochs = 18
steps = 54
val_steps = 10

model= Sequential()

model.add(Convolution2D(32, kernel_size=(3, 3), strides=(1, 1), input_shape=(100,100,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Convolution2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(OutputNeurons, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
model.fit_generator(
                    training_set,
                    steps_per_epoch=steps,
                    epochs=epochs,
                    validation_data=test_set,
                    validation_steps=val_steps)

EndTime=time.time()
print("Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')


C:\Users\clare\AppData\Local\Temp/ipykernel_23920/3415127794.py:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/18
54/54 [==============================] - 20s 362ms/step - loss: 11.6643 - accuracy: 0.0929 - val_loss: 3.3503 - val_accuracy: 0.1094
Epoch 2/18
54/54 [==============================] - 23s 427ms/step - loss: 2.2484 - accuracy: 0.4065 - val_loss: 2.9366 - val_accuracy: 0.2875
Epoch 3/18
54/54 [==============================] - 23s 422ms/step - loss: 1.4513 - accuracy: 0.6151 - val_loss: 2.6041 - val_accuracy: 0.2750
Epoch 4/18
54/54 [==============================] - 20s 377ms/step - loss: 1.0944 - accuracy: 0.7044 - val_loss: 2.1394 - val_accuracy: 0.4812
Epoch 5/18
54/54 [==============================] - 22s 403ms/step - loss: 0.7737 - accuracy: 0.7938 - val_loss: 2.1876 - val_accuracy: 0.4969
Epoch 6/18
54/54 [==============================] - 21s 392ms/step - loss: 0.5678 - accuracy: 0.8259 - val_loss: 1.8610 - val_accuracy: 0.5844
Epoch 7/18
54/54 [==============================] - 23s 431ms/step - loss: 0.5231 - accuracy: 0.8411 - val_loss: 2.1946 - val_accuracy: 0.618

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 10, 10, 128)     

In [7]:
import keras

model.save('finetuned_model.h5')


In [9]:
model_cnn = keras.models.load_model("finetuned_model.h5")

In [18]:
import numpy as np
from keras.preprocessing import image

ImagePath='../Face Images/Final Testing Images/Angela_Bassett/Angela_Bassett_0006.jpg'
test_image=image.load_img(ImagePath,target_size=(100, 100))
test_image=image.img_to_array(test_image)

test_image= np.expand_dims(test_image,axis=0)

result= model_cnn.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('Prediction is: ',PersonMap[np.argmax(result)])

Prediction is:  Angela_Bassett


## Below are the other models that were trained

In [27]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

model= Sequential()

model.add(Convolution2D(32, kernel_size=(4, 4), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Convolution2D(32, kernel_size=(4, 4), strides=(1, 1), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(OutputNeurons, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
model.fit_generator(
                    training_set,
                    steps_per_epoch=50,
                    epochs=18,
                    validation_data=test_set,
                    validation_steps=15)

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')


Epoch 1/18


C:\Users\N00180~1\AppData\Local\Temp/ipykernel_14496/2134870458.py:27: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


50/50 [==============================] - 10s 187ms/step - loss: 7.2437 - accuracy: 0.0952 - val_loss: 3.4198 - val_accuracy: 0.0833
Epoch 2/18
50/50 [==============================] - 9s 182ms/step - loss: 2.7943 - accuracy: 0.2749 - val_loss: 3.0653 - val_accuracy: 0.1646
Epoch 3/18
50/50 [==============================] - 9s 182ms/step - loss: 2.1116 - accuracy: 0.4615 - val_loss: 2.9755 - val_accuracy: 0.2646
Epoch 4/18
50/50 [==============================] - 9s 184ms/step - loss: 1.5999 - accuracy: 0.5817 - val_loss: 2.6959 - val_accuracy: 0.3396
Epoch 5/18
50/50 [==============================] - 9s 183ms/step - loss: 1.2168 - accuracy: 0.6894 - val_loss: 2.6903 - val_accuracy: 0.3313
Epoch 6/18
50/50 [==============================] - 9s 182ms/step - loss: 1.0686 - accuracy: 0.7113 - val_loss: 2.5947 - val_accuracy: 0.4104
Epoch 7/18
50/50 [==============================] - 9s 184ms/step - loss: 0.8694 - accuracy: 0.7652 - val_loss: 2.9204 - val_accuracy: 0.4271
Epoch 8/18
50/50

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 128)       7

In [28]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

model= Sequential()

model.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Convolution2D(32, kernel_size=(4, 4), strides=(1, 1), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Convolution2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(OutputNeurons, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer = 'rmsprop', metrics=["accuracy"])

import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
model.fit_generator(
                    training_set,
                    steps_per_epoch=51,
                    epochs=18,
                    validation_data=test_set,
                    validation_steps=15)

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

Epoch 1/18


C:\Users\N00180~1\AppData\Local\Temp/ipykernel_14496/698799155.py:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


51/51 [==============================] - 10s 190ms/step - loss: 0.4257 - accuracy: 0.0301 - val_loss: 0.1411 - val_accuracy: 0.0646
Epoch 2/18
51/51 [==============================] - 9s 185ms/step - loss: 0.1207 - accuracy: 0.1080 - val_loss: 0.1215 - val_accuracy: 0.1167
Epoch 3/18
51/51 [==============================] - 10s 187ms/step - loss: 0.1036 - accuracy: 0.2044 - val_loss: 0.1082 - val_accuracy: 0.1625
Epoch 4/18
51/51 [==============================] - 10s 186ms/step - loss: 0.0843 - accuracy: 0.3935 - val_loss: 0.1065 - val_accuracy: 0.3229
Epoch 5/18
51/51 [==============================] - 9s 184ms/step - loss: 0.0689 - accuracy: 0.5390 - val_loss: 0.1084 - val_accuracy: 0.2833
Epoch 6/18
51/51 [==============================] - 9s 183ms/step - loss: 0.0584 - accuracy: 0.6182 - val_loss: 0.0975 - val_accuracy: 0.3583
Epoch 7/18
51/51 [==============================] - 10s 185ms/step - loss: 0.0484 - accuracy: 0.6974 - val_loss: 0.0758 - val_accuracy: 0.5188
Epoch 8/18
51

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 128)       7

In [4]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

model= Sequential()

model.add(Convolution2D(32, kernel_size=(3, 3), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Convolution2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(OutputNeurons, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
model.fit_generator(
                    training_set,
                    steps_per_epoch=57,
                    epochs=20,
                    validation_data=test_set,
                    validation_steps=10)

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

C:\Users\N00180~1\AppData\Local\Temp/ipykernel_19240/2381697565.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
57/57 [==============================] - 14s 236ms/step - loss: 5.5586 - accuracy: 0.0643 - val_loss: 3.4167 - val_accuracy: 0.0437
Epoch 2/20
57/57 [==============================] - 14s 239ms/step - loss: 2.7814 - accuracy: 0.2433 - val_loss: 2.7182 - val_accuracy: 0.2844
Epoch 3/20
57/57 [==============================] - 14s 241ms/step - loss: 1.6235 - accuracy: 0.5662 - val_loss: 2.0359 - val_accuracy: 0.4688
Epoch 4/20
57/57 [==============================] - 14s 241ms/step - loss: 1.0615 - accuracy: 0.7073 - val_loss: 1.8682 - val_accuracy: 0.4938
Epoch 5/20
57/57 [==============================] - 14s 237ms/step - loss: 0.7365 - accuracy: 0.8007 - val_loss: 1.5720 - val_accuracy: 0.6250
Epoch 6/20
57/57 [==============================] - 14s 242ms/step - loss: 0.6136 - accuracy: 0.8276 - val_loss: 1.5267 - val_accuracy: 0.6625
Epoch 7/20
57/57 [==============================] - 14s 248ms/step - loss: 0.4876 - accuracy: 0.8627 - val_loss: 1.2558 - val_accuracy: 0.6625

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 128)       7

In [11]:

from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

model= Sequential()

model.add(Convolution2D(32, 
                        kernel_size=(5, 5), 
                        strides=(1, 1), 
                        input_shape=(64,64,3), 
                        activation='relu')
         )

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Convolution2D(64, 
                        kernel_size=(5, 5), 
                        strides=(1, 1), 
                        activation='relu')
         )


model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(OutputNeurons, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
model.fit_generator(
                    training_set,
                    steps_per_epoch=51,
                    epochs=18,
                    validation_data=test_set,
                    validation_steps=15)

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

Epoch 1/18


C:\Users\N00180~1\AppData\Local\Temp/ipykernel_19240/324203942.py:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


51/51 [==============================] - 13s 254ms/step - loss: 10.6259 - accuracy: 0.1344 - val_loss: 3.5249 - val_accuracy: 0.1167
Epoch 2/18
51/51 [==============================] - 11s 217ms/step - loss: 2.4066 - accuracy: 0.3947 - val_loss: 2.6151 - val_accuracy: 0.3333
Epoch 3/18
51/51 [==============================] - 11s 215ms/step - loss: 1.4151 - accuracy: 0.6372 - val_loss: 2.4719 - val_accuracy: 0.4125
Epoch 4/18
51/51 [==============================] - 11s 215ms/step - loss: 1.0200 - accuracy: 0.7311 - val_loss: 2.2673 - val_accuracy: 0.4854
Epoch 5/18
51/51 [==============================] - 11s 209ms/step - loss: 0.7289 - accuracy: 0.8005 - val_loss: 2.4901 - val_accuracy: 0.5167
Epoch 6/18
51/51 [==============================] - 11s 214ms/step - loss: 0.7110 - accuracy: 0.8060 - val_loss: 2.6664 - val_accuracy: 0.4938
Epoch 7/18
51/51 [==============================] - 11s 210ms/step - loss: 0.6866 - accuracy: 0.8171 - val_loss: 2.2605 - val_accuracy: 0.5417
Epoch 8/1

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 60, 60, 32)        2432      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 64)        51264     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 10816)             0         
                                                                 
 dense_3 (Dense)             (None, 64)               